In [1]:
# Make an environment using relevant requirements.txt to satisfy all of the requirements

In [7]:
# Imports for the code

import json
import chromadb
from chromadb import EmbeddingFunction
from chromadb.utils import embedding_functions

In [12]:
# Here we are reading our dataset file

# Specify the path to your JSON file
json_file_path = "G:/All Flutter Applications/NLP with transformers project/chatbot_ic/lib/backend/data/data_json/2014pubmed.json"

# Open the JSON file and load the data
with open(json_file_path, 'r') as json_file:
    dataset = json.load(json_file)

# Print the loaded dataset
print(len(dataset))

1757


In [9]:
client = chromadb.PersistentClient(path="G:/All Flutter Applications/NLP with transformers project/chatbot_ic/lib/backend/data/chroma_data")

In [11]:
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="multi-qa-MiniLM-L6-cos-v1")
my_collection = client.create_collection("2014pubmed", embedding_function=sentence_transformer_ef)

In [13]:
# Extract data from the dataset and store it in the collection
my_collection.add(
    ids=[str(entry['PMID']) for entry in dataset],
    documents=[entry['Abstract'] for entry in dataset],
    metadatas=[
        {'title': entry['Title'], 'author': entry['Author']} for entry in dataset
    ],
)

In [16]:
# checking collection
my_collection.peek()

{'ids': ['22353228',
  '22442254',
  '22508756',
  '22528289',
  '22653808',
  '22907885',
  '22965837',
  '22978709',
  '23019071',
  '23121479'],
 'embeddings': [[0.009692050516605377,
   -0.03641953691840172,
   0.014187151566147804,
   -0.02408846840262413,
   -0.017961081117391586,
   -0.002300935797393322,
   -0.05764165148139,
   0.03233708068728447,
   -0.025178086012601852,
   0.003926010802388191,
   0.0342671163380146,
   -0.0503687746822834,
   -0.058644283562898636,
   0.023324890062212944,
   -0.10569249838590622,
   0.04413844272494316,
   -0.030011804774403572,
   -0.005946860648691654,
   0.021601738408207893,
   -0.045009978115558624,
   -0.00983112957328558,
   0.06131337583065033,
   0.09513452649116516,
   0.03018476814031601,
   0.03599819168448448,
   0.012243944220244884,
   -0.0018987251678481698,
   -0.04997736215591431,
   0.06191946566104889,
   0.016994353383779526,
   0.03270653262734413,
   0.05835413560271263,
   0.003499669721350074,
   -0.0439154691994

In [17]:
# choosing a pmid to inspect
pmid_to_inspect = '22353228'
retrieved_data = my_collection.get(ids=[pmid_to_inspect])

# Print the retrieved data
print(retrieved_data)

{'ids': ['22353228'], 'embeddings': None, 'metadatas': [{'author': 'Susan E Caudle', 'title': 'Nonverbal cognitive development in children with cochlear implants: relationship between the Mullen Scales of Early Learning and later performance on the Leiter International Performance Scales-Revised'}], 'documents': ['Methodologically, longitudinal assessment of cognitive development in young children has proven difficult because few measures span infancy through school age. This matter is further complicated when the child presents with a sensory deficit such as hearing loss. Few measures are validated in this population, and children who are evaluated for cochlear implantation are often reevaluated annually. The authors sought to evaluate the predictive validity of subscales of the Mullen Scales of Early Learning (MSEL) on Leiter International Performance Scales-Revised (LIPS-R) Full-Scale IQ scores. To further elucidate the relationship of these two measures, comparisons were also made 

In [18]:
client.list_collections()

[Collection(name=2014pubmed), Collection(name=2013pubmed)]

In [20]:
collection_2013 = client.get_collection("2013pubmed")

In [23]:
# check on 2013 collection
# Choose a specific document (e.g., using its PMID)
pmid_to_inspect = '25529585'

retrieved_data = collection_2013.get(ids=[pmid_to_inspect], include=['documents', 'embeddings'])

# Print the retrieved data
print(retrieved_data)

{'ids': ['25529585'], 'embeddings': [[0.021833255887031555, -0.020088283345103264, -0.10449198633432388, -0.018440494313836098, 0.06183214858174324, -0.041885316371917725, -0.04321591556072235, 0.07629963010549545, 0.07828796654939651, -0.01864631101489067, -0.039375413209199905, 0.006678316276520491, 0.015885861590504646, 0.0036805106792598963, -0.09417834877967834, -0.05488986149430275, 0.0250967126339674, -0.015076345764100552, -0.030408238992094994, 0.01756281591951847, 0.011605939827859402, -0.007470846641808748, 0.020051274448633194, 0.06507459282875061, -0.039154790341854095, -0.013811176642775536, 0.02269914001226425, -0.016824886202812195, -0.03018326684832573, 0.031918857246637344, 0.014807751402258873, 0.09474042803049088, 0.020402416586875916, -0.022352755069732666, -0.006744012236595154, 0.02220703847706318, 0.0640922337770462, -0.05050061270594597, -0.1019841879606247, 0.04880918189883232, 0.00041741979657672346, 0.03870804235339165, 0.09585507214069366, 0.019260978326201